# Imports and Settings

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os
from scipy.spatial.distance import squareform, cdist
import time
from numpy.testing import *

In [2]:
from Code.DataGeneration.printer import ProgressTimer
from Code.DataGeneration.saver import create_path
from Code.DataGeneration.transform import get_spherical, change_base, get_input_data

In [3]:
path_to_db = "./Dataset/iso17/reference.db"

In [4]:
from ase.db import connect

molecules = []
energies = []
with connect(path_to_db) as conn:
    for row in conn.select():
        molecules.append(np.hstack((row['numbers'].reshape((19, 1)), row['positions'])))
        energies.append(row['total_energy'])

In [5]:
np.array(molecules).shape

(404000, 19, 4)

# Run Calculations

In [ ]:
start = time.time()
network_in = np.array(get_input_data(molecules))
print('time: {}'.format(time.time()-start))

0.0  % ---  70.7
0.0  % ---  73.2
0.1  % ---  79.9
0.1  % ---  76.8
0.1  % ---  74.3
0.1  % ---  70.9
0.2  % ---  69.4
0.2  % ---  67.5
0.2  % ---  66.1
0.2  % ---  64.9
0.3  % ---  64.2
0.3  % ---  64.7
0.3  % ---  63.8
0.3  % ---  63.6
0.4  % ---  62.9
0.4  % ---  63.2
0.4  % ---  62.7
0.4  % ---  62.0
0.5  % ---  61.7
0.5  % ---  61.5
0.5  % ---  61.7
0.5  % ---  61.2
0.6  % ---  61.0
0.6  % ---  60.8
0.6  % ---  60.6
0.6  % ---  60.8
0.7  % ---  60.8
0.7  % ---  60.4
0.7  % ---  60.4
0.7  % ---  60.6
0.8  % ---  60.5
0.8  % ---  60.5
0.8  % ---  60.5
0.8  % ---  60.5
0.9  % ---  60.6
0.9  % ---  60.3
0.9  % ---  60.1
0.9  % ---  60.0
1.0  % ---  60.0
1.0  % ---  60.2
1.0  % ---  60.1
1.0  % ---  59.9
1.1  % ---  59.6
1.1  % ---  59.6
1.1  % ---  59.4
1.1  % ---  59.4
1.2  % ---  59.3
1.2  % ---  59.4
1.2  % ---  59.5
1.2  % ---  59.7
1.3  % ---  60.1
1.3  % ---  60.2
1.3  % ---  60.1
1.3  % ---  60.0
1.4  % ---  59.9
1.4  % ---  59.8
1.4  % ---  59.8
1.4  % ---  59.9
1.5  % ---  59

In [ ]:
network_in.shape

In [ ]:
len(energies)

## Save arrays to file

In [ ]:
data_path = './Dataset/c702h10_X'
label_path = './Dataset/c702h10_Y'

In [ ]:
np.save(data_path, network_in)
np.save(label_path, energies)

# Testing

## Test Functions

In [ ]:
def test_get_spherical():
    test_positions = np.array([[0, 1, 2],
                               [1, 1, 1],
                               [-1, 2, 1]])
    val_result = np.array([[1/np.sqrt(5), np.cos(np.arccos(2/np.sqrt(5))),
                            np.cos(np.pi/2), np.sin(np.pi/2)],
                           [1/np.sqrt(3), np.cos(np.arccos(1/np.sqrt(3))),
                            np.cos(np.arctan(1)), np.sin(np.arctan(1))],
                           [1/np.sqrt(6), np.cos(np.arccos(1/np.sqrt(6))),
                            np.cos(np.arctan(-2) + np.pi), np.sin(np.arctan(-2) + np.pi)]])
    assert_array_almost_equal(val_result, get_spherical(test_positions)) 
    print('pass!')

In [ ]:
def test_change_base():
    test_positions = np.array([[0, 1, 2],
                               [1, 1, 1],
                               [-1, 2, 1]])
    x = np.array([1, 1, 0])
    y = np.array([0, 0, 1])
    z = np.array([2, 1, 3])
    val_result = np.array([[-7.,-13.,4.],
                           [-8.,-17.,5.],
                           [-4., -8.,2.]])
    o = np.array([-1, 4, 3])
    assert_array_almost_equal(val_result, change_base(test_positions, x, y, z, o))
    print('pass!')

In [ ]:
def test_input_data():
    # load molecule and sort
    test_mol = molecules[0]
    test_mol = test_mol[test_mol[:,0].argsort(kind='mergesort')]
    results = np.zeros((19, 72))
    positions = test_mol[:, 1:]
    # H-part
    for i in range(10):
        dists = cdist(positions[np.newaxis, i], positions[10:])
        one, two = dists.argsort().reshape(9)[:2] + 10
        zero = i
        x = positions[one] - positions[zero]
        z = np.cross(x, positions[two] - positions[zero])
        y = np.cross(z, x)
        x /= np.linalg.norm(x)
        y /= np.linalg.norm(y)
        z /= np.linalg.norm(z)
        others = np.vstack((test_mol[:i], test_mol[i+1:]))
        ch_b_others = change_base(others[:, 1:], x, y, z, positions[zero])
        sph_others = get_spherical(ch_b_others)
        ids = sph_others[:, 0].argsort(kind='mergesort')
        sph_others = sph_others[ids]
        others = others[ids]
        sph_others = sph_others[others[:, 0].argsort(kind='mergesort')]
        results[i] = sph_others.reshape(72)
    # C-part
    for i in range(10, 17):
        others = np.vstack((test_mol[:i], test_mol[i+1:]))
        neighs = others[10:]
        dists = cdist(positions[np.newaxis, i], neighs[:, 1:])
        one_id, two_id = dists.argsort().reshape(8)[:2]
        one, two = neighs[one_id, 1:], neighs[two_id, 1:]
        zero =test_mol[i, 1:]
        x = one - zero
        z = np.cross(x, two - zero)
        y = np.cross(z, x)
        x /= np.linalg.norm(x)
        y /= np.linalg.norm(y)
        z /= np.linalg.norm(z)
        ch_b_others = change_base(others[:, 1:], x, y, z, zero)
        sph_others = get_spherical(ch_b_others)
        ids = sph_others[:, 0].argsort(kind='mergesort')
        sph_others = sph_others[ids]
        others = others[ids]
        sph_others = sph_others[others[:, 0].argsort(kind='mergesort')]
        results[i] = sph_others.reshape(72)
    # O-part
    for i in range(17, 19):
        others = np.vstack((test_mol[:i], test_mol[i+1:]))
        neighs = others[10:]
        dists = cdist(positions[np.newaxis, i], neighs[:, 1:])
        one_id, two_id = dists.argsort().reshape(8)[:2]
        one, two = neighs[one_id, 1:], neighs[two_id, 1:]
        zero =test_mol[i, 1:]
        x = one - zero
        z = np.cross(x, two - zero)
        y = np.cross(z, x)
        x /= np.linalg.norm(x)
        y /= np.linalg.norm(y)
        z /= np.linalg.norm(z)
        ch_b_others = change_base(others[:, 1:], x, y, z, zero)
        sph_others = get_spherical(ch_b_others)
        ids = sph_others[:, 0].argsort(kind='mergesort')
        sph_others = sph_others[ids]
        others = others[ids]
        sph_others = sph_others[others[:, 0].argsort(kind='mergesort')]
        results[i] = sph_others.reshape(72)
    test_result = np.array(get_input_data(test_mol[np.newaxis])).reshape(19, 72)
    
    assert_array_almost_equal(test_result, results)
    print('pass!')

## Run Tests

In [ ]:
test_get_spherical()

In [ ]:
test_change_base()

In [ ]:
test_input_data()